In [1]:
# libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn import preprocessing
import sys
import json
from utils import make_dir
import matplotlib.pyplot as plt
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
# from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler


/home/mtech_22/anaconda3/envs/ashwanth/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from constants import *

# CONSTANTS
COMET_SCORE = 'comet_score'
BLEU_SCORE = 'bleu_score'
NO_OF_TOKENS_IN_QUERY = 'no_of_tokens_in_query'
NO_OF_TOKENS_IN_SRC_SENT = 'no_of_tokens_in_src_sent'
NO_OF_TOKENS_IN_DST_SENT = 'no_of_tokens_in_dst_sent'
LABSE_SCORE_QUERY_SRC = 'labse_score_query_src'
LABSE_SCORE_QUERY_DST = 'labse_score_query_dst'
LABSE_SCORE_SRC_DST = 'labse_score_src_dst'
CHRF_SCORE = 'chrf_score'

COMET_QE_QUERY_SRC_SCORE = 'comet_qe_query_src_score'
COMET_QE_QUERY_DST_SCORE = 'comet_qe_query_dst_score'
COMET_QE_SRC_DST_SCORE = 'comet_qe_src_dst_score'
SRC_DST_PPL = 'src_dst_ppl'
SRC_DST_QUERY_PPL = 'src_dst_query_ppl'

IN22_OTHER_SOURCES = 'in22_other_sources'
SAMANANTAR = 'samanantar'
FLORES = 'flores'

SAVED_MODELS = 'saved_models'
DATASET_TRAIN = 'dataset_train'
DATASET_TEST = 'dataset_test'

TOKEN_RATIO_QUERY_TO_SRC = 'token_ratio_query_to_src'
TOKEN_RATIO_SRC_TO_DST = 'token_ratio_src_to_dst'

In [3]:
features = [NO_OF_TOKENS_IN_QUERY, 
            NO_OF_TOKENS_IN_SRC_SENT, 
            NO_OF_TOKENS_IN_DST_SENT,
            LABSE_SCORE_QUERY_SRC, 
            LABSE_SCORE_QUERY_DST, 
            LABSE_SCORE_SRC_DST,
            CHRF_SCORE, 
            COMET_QE_QUERY_SRC_SCORE, 
            COMET_QE_QUERY_DST_SCORE, 
            COMET_QE_SRC_DST_SCORE,
            # SRC_PPL, 
            # DST_PPL, 
            SRC_DST_PPL, 
            SRC_DST_QUERY_PPL] 

Features = [LABSE_SCORE_QUERY_SRC, LABSE_SCORE_QUERY_DST, LABSE_SCORE_SRC_DST,
            CHRF_SCORE, 
            COMET_QE_QUERY_SRC_SCORE, COMET_QE_QUERY_DST_SCORE, COMET_QE_QUERY_DST_SCORE,
            SRC_DST_PPL, SRC_DST_QUERY_PPL] 

Filter negative comet scores.

Increase train dataset;

Ignore number of tokens;

All languages; All directions;

### Inputs

In [4]:
training_source = SAMANANTAR
testing_source = FLORES
src_lang = HIN_DEVA
dst_lang = ENG_LATN
approach = 'minmaxscalar'

### Load and process data

In [5]:
# Training dataset is created using bloom.ipynb file. (Refer to get_prompt_scores function).
dataset_path = '{}/{}_{}_{}.csv'.format(DATASET_TRAIN, training_source, src_lang, dst_lang)
dataset = pd.read_csv(dataset_path)
# dataset = dataset[dataset[COMET_SCORE] >= 0]
# dataset[TOKEN_RATIO_QUERY_TO_SRC] = dataset[NO_OF_TOKENS_IN_QUERY].div(dataset[NO_OF_TOKENS_IN_SRC_SENT], fill_value=1)
# dataset[TOKEN_RATIO_SRC_TO_DST] = dataset[NO_OF_TOKENS_IN_SRC_SENT].div(dataset[NO_OF_TOKENS_IN_DST_SENT], fill_value=1)
dataset = dataset.drop(['qid_tmp', 'index_tmp'], axis=1)
dataset.replace([np.inf], 99999, inplace=True)
dataset

,qid,index,no_of_tokens_in_query,no_of_tokens_in_src_sent,no_of_tokens_in_dst_sent,labse_score_query_src,labse_score_query_dst,labse_score_src_dst,chrf_score,comet_qe_query_src_score,comet_qe_query_dst_score,comet_qe_src_dst_score,src_ppl,dst_ppl,src_dst_ppl,src_dst_query_ppl,comet_score,bleu_score
0,0,2979793,54,27,25,0.50,0.39,0.82,34.09,0.00,0.49,0.71,11.01,11.61,7.98,6.85,0.7278,30.87
1,0,8510795,54,28,25,0.51,0.39,0.87,33.80,0.00,0.49,0.69,11.04,11.61,7.67,6.63,0.7382,31.70
2,0,4533373,54,22,18,0.50,0.24,0.86,34.37,0.20,0.31,0.74,12.36,8.97,5.95,5.84,0.7245,30.14
3,0,4310762,54,27,25,0.39,0.29,0.79,25.88,0.00,0.00,0.30,10.43,22.35,9.59,7.81,0.7382,31.70
4,0,3709762,54,23,15,0.52,0.38,0.78,28.34,0.00,0.00,0.56,12.70,14.81,8.23,6.86,0.7072,37.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99695,996,9574828,12,32,34,0.41,0.23,0.76,16.69,0.25,0.01,0.30,10.55,10.27,7.17,8.23,0.6675,18.75
99696,996,6374803,12,18,16,0.46,0.29,0.86,24.41,0.80,0.29,0.42,9.16,16.28,5.14,6.11,0.5167,7.77
99697,996,4437987,12,18,16,0.46,0.29,0.86,24.41,0.81,0.29,0.44,8.73,16.28,4.97,5.94,0.5167,7.77
99698,996,4266589,12,11,11,0.39,0.33,0.87,23.70,0.18,0.00,0.01,71.21,99.21,43.00,24.41,0.6412,4.21


In [6]:
# create feature variables and y
df = dataset.copy()
X = df.drop(['comet_score', 'bleu_score'], axis=1)
y = df[['comet_score']]

# create train/val/test dataset
X_train, X_rem, y_train, y_rem = train_test_split(X, y, test_size=0.2, random_state=10)
X_val, X_test, y_val, y_test = train_test_split(X_rem, y_rem, test_size=0.5, random_state=10)

In [7]:
# pick only the necessary features
X_train = X_train[features]
X_val = X_val[features]
# X_test = X_test[features]

In [8]:
X_train

,no_of_tokens_in_query,no_of_tokens_in_src_sent,no_of_tokens_in_dst_sent,labse_score_query_src,labse_score_query_dst,labse_score_src_dst,chrf_score,comet_qe_query_src_score,comet_qe_query_dst_score,comet_qe_src_dst_score,src_dst_ppl,src_dst_query_ppl
92935,33,24,21,0.46,0.30,0.85,28.44,0.30,0.00,0.26,11.20,10.66
86562,33,10,8,0.63,0.47,0.84,26.00,0.27,0.04,0.68,12.34,8.91
99073,16,14,14,0.50,0.17,0.44,30.64,0.00,0.00,0.18,15.41,11.52
21487,11,15,16,0.53,0.35,0.85,24.41,0.06,0.14,0.61,5.88,8.29
21700,19,13,8,0.51,0.39,0.81,41.33,0.30,0.00,0.20,12.55,12.22
...,...,...,...,...,...,...,...,...,...,...,...,...
9372,22,47,33,0.50,0.30,0.79,17.84,0.20,0.23,0.53,6.04,6.83
93553,29,14,15,0.38,0.30,0.85,17.94,0.00,0.09,0.28,8.41,8.98
50496,12,18,11,0.64,0.51,0.90,23.34,0.45,0.36,0.78,5.40,6.69
94735,21,27,29,0.54,0.44,0.93,15.25,0.32,0.39,0.63,4.82,6.05


In [9]:
X_train_feature_tmp = X_train

In [10]:
# Standardizing data
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)
# X_test = scaler.transform(X_test)

y_scalar = MinMaxScaler()
y_scalar.fit(y_train)
y_train = y_scalar.transform(y_train)
y_val = y_scalar.transform(y_val)
# y_test = y_scalar.transform(y_test)

In [11]:
len(features) == len(X_val[0])

True

In [12]:
# set seed
def set_seed():
    torch.manual_seed(42)
    np.random.seed(42)

set_seed()

In [13]:
"""
NO_OF_TOKENS_IN_QUERY = 'no_of_tokens_in_query'
NO_OF_TOKENS_IN_SRC_SENT = 'no_of_tokens_in_src_sent'
NO_OF_TOKENS_IN_DST_SENT = 'no_of_tokens_in_dst_sent'
LABSE_SCORE_QUERY_SRC = 'labse_score_query_src'
LABSE_SCORE_QUERY_DST = 'labse_score_query_dst'
LABSE_SCORE_SRC_DST = 'labse_score_src_dst'
CHRF_SCORE = 'chrf_score'
COMET_QA_SCORE = 'comet_qa_score'
"""
# visualize the data
# sns.scatterplot(x=COMET_QA_SCORE, y=COMET_SCORE, data=dataset)

"\nNO_OF_TOKENS_IN_QUERY = 'no_of_tokens_in_query'\nNO_OF_TOKENS_IN_SRC_SENT = 'no_of_tokens_in_src_sent'\nNO_OF_TOKENS_IN_DST_SENT = 'no_of_tokens_in_dst_sent'\nLABSE_SCORE_QUERY_SRC = 'labse_score_query_src'\nLABSE_SCORE_QUERY_DST = 'labse_score_query_dst'\nLABSE_SCORE_SRC_DST = 'labse_score_src_dst'\nCHRF_SCORE = 'chrf_score'\nCOMET_QA_SCORE = 'comet_qa_score'\n"

In [14]:
# # features = [NO_OF_TOKENS_IN_QUERY, NO_OF_TOKENS_IN_SRC_SENT, NO_OF_TOKENS_IN_DST_SENT, 
# #                 LABSE_SCORE_QUERY_SRC, LABSE_SCORE_QUERY_DST, LABSE_SCORE_SRC_DST,
# #                 CHRF_SCORE, COMET_QA_SCORE ]

# from sklearn.preprocessing import StandardScaler, MinMaxScaler 
# scaled_features =  X_train[features]
# x_scalar = MinMaxScaler().fit(scaled_features.values)
# scaled_features = x_scalar.transform(scaled_features.values)
# X_train[features] = scaled_features
# X_val[features] = x_scalar.transform(X_val[features].values)
# X_test[features] = x_scalar.transform(X_test[features].values)

# scaled_features = y_train[[COMET_SCORE]]
# y_scalar = MinMaxScaler().fit(scaled_features.values)
# scaled_features = y_scalar.transform(scaled_features.values)
# y_train[COMET_SCORE] = scaled_features
# y_val[COMET_SCORE] = y_scalar.transform(y_val[[COMET_SCORE]].values)
# y_test[COMET_SCORE] = y_scalar.transform(y_test[[COMET_SCORE]].values)

In [15]:
# create a regression model
model = LinearRegression()

# define and train the Ridge model with regularization parameter alpha
# from sklearn.linear_model import Ridge
# alpha = 0.2
# model = Ridge(alpha=alpha)
# ridge.fit(X_train, y_train)

# fit the model
model.fit(X_train, y_train)

# make predictions
y_val_pred = model.predict(X_val)

# model evaluation
print('mean_squared_error : ', mean_squared_error(y_val, y_val_pred))
print('mean_absolute_error : ', mean_absolute_error(y_val, y_val_pred))

mean_squared_error :  0.02724106518106071
mean_absolute_error :  0.11171779930411066


In [16]:
def get_importance_coeff():
    importances = pd.DataFrame(data={
        'Attribute': X_train_feature_tmp.columns,
        'Importance': model.coef_[0]
    })
    importances = importances.sort_values(by='Importance', ascending=False)
    importances.to_csv('importances.csv')

    plt.bar(x=importances['Attribute'], height=importances['Importance'], color='#087E8B')
    plt.title('Feature importances obtained from coefficients', size=20)
    plt.xticks(rotation='vertical')
    plt.show()

In [17]:
# get_importance_coeff()

In [18]:
features

['no_of_tokens_in_query',
 'no_of_tokens_in_src_sent',
 'no_of_tokens_in_dst_sent',
 'labse_score_query_src',
 'labse_score_query_dst',
 'labse_score_src_dst',
 'chrf_score',
 'comet_qe_query_src_score',
 'comet_qe_query_dst_score',
 'comet_qe_src_dst_score',
 'src_dst_ppl',
 'src_dst_query_ppl']

In [19]:
a = model.coef_[0]

In [20]:
model.intercept_

array([0.55423064])

In [21]:
for item1, item2 in zip(a, features):
    print('{}, {}'.format(item1, item2))

-0.05402462102432366, no_of_tokens_in_query
-0.3005217690636133, no_of_tokens_in_src_sent
-0.2523368338001344, no_of_tokens_in_dst_sent
0.3154435288405524, labse_score_query_src
-0.22342896109389568, labse_score_query_dst
0.26471941355350936, labse_score_src_dst
0.048395274648092595, chrf_score
-0.04704098150143904, comet_qe_query_src_score
0.02895891177766402, comet_qe_query_dst_score
-0.002447316162636208, comet_qe_src_dst_score
0.3044537080185562, src_dst_ppl
-0.6314457332063026, src_dst_query_ppl


In [22]:
# mean_squared_error :  0.027887389493700655
# mean_absolute_error :  0.11378171350541372

In [23]:
# TODO: Didn't use test data from the training dataset

### Get prompt score for FLORES devtest

In [24]:
# load the test dataset
test_data_path = '{}/{}_{}_{}_{}.csv'.format(DATASET_TEST, training_source, testing_source, src_lang, dst_lang)
X_test = pd.read_csv(test_data_path) 
# X_test[TOKEN_RATIO_QUERY_TO_SRC] = X_test[NO_OF_TOKENS_IN_QUERY].div(X_test[NO_OF_TOKENS_IN_SRC_SENT], fill_value=1)
# X_test[TOKEN_RATIO_SRC_TO_DST] = X_test[NO_OF_TOKENS_IN_SRC_SENT].div(X_test[NO_OF_TOKENS_IN_DST_SENT], fill_value=1)
X_test.replace([np.inf], 99999, inplace=True)
scaled_features =  X_test[features]
scaled_features = scaler.transform(scaled_features.values)
X_test[features] = scaled_features
X_test

/home/mtech_22/anaconda3/envs/ashwanth/lib/python3.8/site-packages/sklearn/base.py:409: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


,qid,index,no_of_tokens_in_query,no_of_tokens_in_src_sent,no_of_tokens_in_dst_sent,labse_score_query_src,labse_score_query_dst,labse_score_src_dst,chrf_score,comet_qe_query_src_score,comet_qe_query_dst_score,comet_qe_src_dst_score,src_ppl,dst_ppl,src_dst_ppl,src_dst_query_ppl
0,0,6604420,0.258621,0.034483,0.039648,0.529412,0.386139,0.873786,0.426817,0.70,0.459184,0.848485,28.07,23.19,0.000463,0.064825
1,0,2394465,0.258621,0.017241,0.013216,0.578431,0.485149,0.912621,0.345721,0.52,0.071429,0.989899,37.00,33.91,0.000700,0.085636
2,0,3302386,0.258621,0.017241,0.022026,0.568627,0.504950,0.864078,0.244014,0.66,0.183673,0.868687,55.60,29.19,0.000839,0.093589
3,0,7981177,0.258621,0.041379,0.030837,0.568627,0.465347,0.922330,0.229440,0.56,0.530612,0.747475,14.60,23.09,0.000291,0.061536
4,0,5017151,0.258621,0.027586,0.035242,0.529412,0.415842,0.834951,0.201333,0.96,0.244898,0.616162,20.75,23.71,0.000468,0.067934
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101195,1011,1554348,0.448276,0.079310,0.118943,0.480392,0.514851,0.825243,0.267437,0.00,0.285714,0.323232,9.61,7.62,0.000257,0.030738
101196,1011,6047900,0.448276,0.162069,0.185022,0.480392,0.376238,0.844660,0.252446,0.01,0.397959,0.676768,14.67,8.98,0.000298,0.034027
101197,1011,4584651,0.448276,0.117241,0.149780,0.539216,0.415842,0.834951,0.298251,0.00,0.142857,0.494949,8.39,15.44,0.000208,0.025774
101198,1011,1333535,0.448276,0.082759,0.096916,0.431373,0.356436,0.864078,0.259109,0.00,0.000000,0.303030,29.21,13.32,0.000468,0.038931


In [25]:
# predict for test data
y_test_pred = model.predict(X_test[features])
X_test['prompt_score'] = y_test_pred
X_test

/home/mtech_22/anaconda3/envs/ashwanth/lib/python3.8/site-packages/sklearn/base.py:402: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


,qid,index,no_of_tokens_in_query,no_of_tokens_in_src_sent,no_of_tokens_in_dst_sent,labse_score_query_src,labse_score_query_dst,labse_score_src_dst,chrf_score,comet_qe_query_src_score,comet_qe_query_dst_score,comet_qe_src_dst_score,src_ppl,dst_ppl,src_dst_ppl,src_dst_query_ppl,prompt_score
0,0,6604420,0.258621,0.034483,0.039648,0.529412,0.386139,0.873786,0.426817,0.70,0.459184,0.848485,28.07,23.19,0.000463,0.064825,0.790080
1,0,2394465,0.258621,0.017241,0.013216,0.578431,0.485149,0.912621,0.345721,0.52,0.071429,0.989899,37.00,33.91,0.000700,0.085636,0.785452
2,0,3302386,0.258621,0.017241,0.022026,0.568627,0.504950,0.864078,0.244014,0.66,0.183673,0.868687,55.60,29.19,0.000839,0.093589,0.749921
3,0,7981177,0.258621,0.041379,0.030837,0.568627,0.465347,0.922330,0.229440,0.56,0.530612,0.747475,14.60,23.09,0.000291,0.061536,0.799128
4,0,5017151,0.258621,0.027586,0.035242,0.529412,0.415842,0.834951,0.201333,0.96,0.244898,0.616162,20.75,23.71,0.000468,0.067934,0.745606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101195,1011,1554348,0.448276,0.079310,0.118943,0.480392,0.514851,0.825243,0.267437,0.00,0.285714,0.323232,9.61,7.62,0.000257,0.030738,0.732221
101196,1011,6047900,0.448276,0.162069,0.185022,0.480392,0.376238,0.844660,0.252446,0.01,0.397959,0.676768,14.67,8.98,0.000298,0.034027,0.725911
101197,1011,4584651,0.448276,0.117241,0.149780,0.539216,0.415842,0.834951,0.298251,0.00,0.142857,0.494949,8.39,15.44,0.000208,0.025774,0.756341
101198,1011,1333535,0.448276,0.082759,0.096916,0.431373,0.356436,0.864078,0.259109,0.00,0.000000,0.303030,29.21,13.32,0.000468,0.038931,0.753218


In [26]:
# clean the test outputs
X_test['prompt_score'] = X_test['prompt_score'].apply(lambda x: round(x, 4))
X_test = X_test.sort_values(by=['qid'])

# sort the predicted scores 
result = {}
for i, row in X_test.iterrows():
    qid, index, pred_comet_score = row['qid'], row['index'], row['prompt_score']
    # print(qid, index, pred_comet_score)
    qid, index = int(qid), int(index)
    if qid not in result:
        result[qid] = []
    
    result[qid].append({"index": index, "score": pred_comet_score})

# sort based on the predicted prompt score
for qid in list(result.keys()):
    ranking = result[qid]
    ranking.sort(key=lambda x: x['score'], reverse=True)
    result[qid] = ranking

# write score to a JSON file
make_dir('rankings_linear_regression')
with open('rankings_linear_regression/recommendations_{}_{}_{}_{}.json'.format(training_source, testing_source, src_lang, dst_lang), 'w') as f:
    json.dump(result, f, indent=4, ensure_ascii=False)